In [1]:
import torch
import torchvision.models as models

# 加載預訓練的 DenseNet161 模型
model = models.densenet161()

# 定義兩個容器來保存 GAP 的輸入和輸出
gap_inputs = []
gap_outputs = []

def hook(module, input, output):
    # input 是一個包含單個元素的元組，因為前一層的輸出是這一層的輸入
    gap_inputs.append(input[0])  
    gap_outputs.append(output)

# 在 DenseNet 的 feature 模塊的最後進行 hook，假設 'norm5' 是最後的批量正規化層
# Global Average Pooling 層應該接在這個層之後
model.classifier.register_forward_hook(hook)

# 將模型設置為評估模式
model.eval()

# 創建一個隨機輸入張量
input_tensor = torch.randn(1, 3, 224, 224)

# 執行模型，觸發 hook
with torch.no_grad():
    model(input_tensor)

# 打印輸入和輸出的形狀
print("Shape of GAP input:", gap_inputs[-1].shape)
print("Shape of GAP output:", gap_outputs[-1].shape)


Shape of GAP input: torch.Size([1, 2208])
Shape of GAP output: torch.Size([1, 1000])


In [2]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [19]:
import torch
import torch.nn as nn
import torchvision.models as models

class CustomDenseNet(nn.Module):
    def __init__(self, pretrained=True, num_classes=1000):
        super(CustomDenseNet, self).__init__()
        self.features = models.densenet161().features
        # self.reduction = nn.Conv2d(2208, 28, kernel_size=1, stride=1, padding=0)
        self.to_features = nn.Flatten(start_dim=2)
        self.attention = nn.MultiheadAttention(embed_dim=49, num_heads=1, dropout=0.1, batch_first=True)
        self.classifier = nn.Linear(49, 28)
        
    def forward(self, x):
        x = self.features(x)
        # x = self.reduction(x)
        x = self.to_features(x)
        x, _ = self.attention(x, x, x)
        # x = torch.mean(x, dim=2)
        x = self.classifier(x[:, -1, :])
        return x

In [20]:
model = CustomDenseNet()

# 假设有一个输入特征图
input_tensor = torch.randn(1, 3, 224, 224)

# 应用模型
output = model(input_tensor)

print(output.shape)
output[0]

torch.Size([1, 28])


tensor([-0.0751, -0.0257, -0.0277,  0.0296,  0.0184,  0.0611,  0.0310,  0.0181,
         0.0459, -0.0607,  0.0044, -0.0416, -0.0086, -0.1511, -0.0357,  0.1188,
        -0.0304,  0.0905, -0.0124, -0.1595,  0.1295, -0.0427, -0.0671,  0.1681,
         0.0179, -0.0211,  0.0249,  0.0255], grad_fn=<SelectBackward0>)